In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
pd.set_option('max_rows', 500)

from tqdm.notebook import tqdm

# load data

In [2]:
train = pd.read_csv('raw_data/train_data_public.csv', usecols=['text', 'BIO_anno', 'class', 'bank_topic'])
train['id'] = [i for i in range(len(train))]

print(train.shape)
train.head()

(10000, 5)


,text,BIO_anno,class,bank_topic,id
0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0,建设银行,0
1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1,建设银行,1
2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0,建设银行,2
3,孙女士在原恒泰农村合作银行存入50万元，同年9月又存款50万元。2014年，恒泰农村合作银行...,O O O O O B-BANK I-BANK I-BANK I-BANK I-BANK I...,2,NaN,3
4,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2,建设银行,4


In [3]:
test = pd.read_csv('raw_data/test_public.csv')

print(test.shape)
test.head()

(5093, 2)


,id,text
0,0,一个月有不到40笔吗？卡多了也是累赘。再看就不能打电话交保护费保险了。
1,1,光大│福卡13K
2,2,对呀，才四万，又不是很多
3,3,为啥我10万每个月800啊，这个额度分分钟
4,4,3. 债券，一般有多余的钱可以考虑的，还是不错的投资


In [4]:
submit = pd.read_csv('raw_data/submit_example.csv')

print(submit.shape)
submit.head()

(5093, 3)


,id,BIO_anno,class
0,0,O O O O O O O O O O O O O O O O O O O O O O O ...,2
1,1,O O O O O O O O,2
2,2,O O O O O O O O O O O O,2
3,3,O O O O O O O O O O O O O O O O O O O O O,2
4,4,O O O O O O O O O O O O O O O O O O O O O O O ...,2


# NER

In [5]:
from simpletransformers.ner import NERModel, NERArgs

In [6]:
train_data = list()

for _, row in tqdm(train.iterrows()):
    sentence_id = row['id']
    sentence = row['text']
    bios = row['BIO_anno'].split()
    for idx, char in enumerate(sentence):
        res = dict()
        res['sentence_id'] = sentence_id
        res['words'] = char
        res['labels'] = bios[idx]
        train_data.append(res)
        
train_data = pd.DataFrame(train_data)

print(train_data.shape)
train_data.head(100)


(210429, 3)


,sentence_id,words,labels
0,0,交,B-BANK
1,0,行,I-BANK
2,0,1,O
3,0,4,O
4,0,年,O
5,0,用,O
6,0,过,O
7,0,，,O
8,0,半,O
9,0,年,O


In [7]:
labels = [
    'B-BANK',
    'I-BANK',
    'B-PRODUCT',
    'I-PRODUCT',
    'B-COMMENTS_N',
    'I-COMMENTS_N',
    'B-COMMENTS_ADJ',
    'I-COMMENTS_ADJ',
    'O'
]

model_args = NERArgs()
model_args.max_seq_length = 400
model_args.train_batch_size = 8
model_args.num_train_epochs = 3
model_args.fp16 = False
model_args.evaluate_during_training = True

In [8]:
model = NERModel("bert", 
                 "hfl/chinese-bert-wwm-ext",
                 labels=labels,
                 args=model_args)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/

In [9]:
train_df = train_data[train_data['sentence_id'] < 9000]
valid_df = train_data[train_data['sentence_id'] >= 9000]

model.train_model(train_df, eval_data=valid_df)

(3375,
 {'global_step': [1125, 2000, 2250, 3375],
  'precision': [0.848078096400244,
   0.8239067055393586,
   0.8167808219178082,
   0.8505002942907592],
  'recall': [0.8671241422333126,
   0.8814722395508422,
   0.8927011852776044,
   0.9014348097317529],
  'f1_score': [0.8574953732264035,
   0.8517179023508137,
   0.8530551415797317,
   0.8752271350696547],
  'train_loss': [0.1103605329990387,
   0.2675149142742157,
   0.09671870619058609,
   0.05714509263634682],
  'eval_loss': [0.10910721370577812,
   0.10822772925719619,
   0.10991965884529054,
   0.11606662555783986]})

In [10]:
pred, _ = model.predict(test['text'], split_on_space=False)

In [11]:
sub_data = list()

for idx, row in tqdm(test.iterrows()):
    res = dict()
    res['id'] = idx
    res['BIO_anno'] = ' '.join([list(i.values())[0] for i in pred[idx]])
    sub_data.append(res)
    
sub = pd.DataFrame(sub_data)
sub.head()

,id,BIO_anno
0,0,O O O O O O O O O O O O O O O O B-COMMENTS_ADJ...
1,1,B-BANK I-BANK O B-PRODUCT I-PRODUCT O O O
2,2,O O O O O O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ
3,3,O O O O O O O O O O O O O O O O B-COMMENTS_N I...
4,4,O O B-PRODUCT I-PRODUCT O O O O B-COMMENTS_ADJ...


In [12]:
sub.to_csv('ner_result.csv', index=False)

# Classifier

In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [6]:
train['class'].value_counts(dropna=False)

2    9620
0     242
1     138
Name: class, dtype: int64

In [7]:
model_args = ClassificationArgs()
model_args.max_seq_length = 400
model_args.train_batch_size = 8
model_args.num_train_epochs = 3
model_args.fp16 = False

In [8]:
model = ClassificationModel("bert",
                            "hfl/chinese-bert-wwm-ext",
                            num_labels=3,
                            args=model_args)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [9]:
train_df = train[['text', 'class']].copy()
train_df.columns = ['text', 'labels']

In [10]:
model.train_model(train_df)

(3750, 0.17493137988795837)

In [11]:
pred, _ = model.predict(list(test['text']))

In [12]:
sub = pd.read_csv('ner_result.csv')
sub['class'] = pred
sub.head()

,id,BIO_anno,class
0,0,O O O O O O O O O O O O O O O O B-COMMENTS_ADJ...,2
1,1,B-BANK I-BANK O B-PRODUCT I-PRODUCT O O O,2
2,2,O O O O O O O O O O B-COMMENTS_ADJ I-COMMENTS_ADJ,2
3,3,O O O O O O O O O O O O O O O O B-COMMENTS_N I...,2
4,4,O O B-PRODUCT I-PRODUCT O O O O B-COMMENTS_ADJ...,2


In [13]:
sub['class'].value_counts()

2    4998
0      64
1      31
Name: class, dtype: int64

In [14]:
sub.to_csv('baseline.csv', index=False)